# FORESEE - ALP with photon couplings

### Load Libraries 

In [3]:
import numpy as np
import sys
import os
src_path = "/Users/felixkling/Documents/GitHub/FORESEE/"
sys.path.append(src_path)
from src.foresee import Foresee, Utility, Model

## 1. Initialization 

We will use some function from the FORESEE package. So let's load it first.

In [4]:
foresee = Foresee(path=src_path)

## 2. ALP Production via Primakoff Process

The phenomenology of the ALP with $\gamma$-couplings can be described by the following Lagrangian

\begin{equation}
 \mathcal{L} = - \frac{1}{2} m_{a}^2 a^2  - \frac{g}{4}a F^a_{\mu\nu} \tilde F^a_{\mu\nu}
\end{equation}

with the ALP mass $a$ and the coupling parameter $g$ as free parameters. For the search for ALPS at forward experiments we need to know i) the *production rate*, ii) the *lifetime* and iii) possibly the *decay branching fractions* of dark photons as function of those two parameters. All properties are specified in the `Model` class. We initialize it with the name of the model as argument. 

**Production** The Dark Higgs is mainly produced via the Primakoff Process. Following [1806.02348](https://arxiv.org/pdf/1806.02348.pdf) Eq (6), the Primakoff cross section is given by
\begin{equation}
 \frac{d\sigma}{d\log\theta } 
 = \theta \frac{d\sigma}{d\log\theta} = g^2 \frac{\alpha Z^2}{4} \times F(t)^2 \times \frac{p_a^4 \theta^4}{t^2}
\end{equation}
where $p_a \approx E_\gamma$, $t = m_a^4 / (2E_\gamma^2) + E_\gamma^2 \theta^2$ and the form factor is given by $F(t) = a^2t/(1+a^2t)$ for $t<7.39m_e^2$ and $F(t) = 1/(1+t/d)$ for $t>7.39m_e^2$. In the following, we will get the ALP spectrum and save it. 

Let us first define a function `get_spectrum()` that obtains this spectrum.

In [5]:
def get_spectrum(mass, generator="EPOSLHC", energy="14"):
    
    #load photon spectrum
    filepath = src_path+"files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    #define quantities
    alpha, me, ZFe, AFe = 1./137., 0.000511, 26, 56
    a, d  = 111.*ZFe**(-1./3.) / me,  0.164 * AFe**(-2./3.)
    SMXSinBarn, GeV2inBarn = 5, 0.389379e-3
    SMXSinIGeV2 = 13311.9696379 

    # integration boundaries theta 
    logthmin, logthmax, nlogth = -12, 0, 20
    dlogth = (logthmax-logthmin)/float(nlogth)
    
    # prepare output file
    filename_out = energy+"TeV/Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth,logen,wgt in photons:
        # weight == 0
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue
        # finite weight
        xs, xsx = 0, 0
        k = 10**logen
        for ltheta in np.linspace(logthmin+0.5*dlogth,logthmax-0.5*dlogth,nlogth):
            # kinematics
            theta = 10**ltheta
            t = mass**4 / (2.*k**2) + k**2 * theta**2
            #form factor 
            if t<7.39*me*me: ff = a*a*t/(1.+a*a*t)
            else: ff = 1./(1.+t/d)
            #combine
            xs += alpha * ZFe**2 / 4. * ff**2 * k**4 * np.sin(theta)**3 *theta / t**2 * dlogth * np.log(10)

        # print 
        prob = xs / SMXSinIGeV2
        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    #close file
    f.close()

And then loop over a set of masses, and calculate it

In [6]:
masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,0,20+1)]]
for mass in masses: 
    get_spectrum(mass=mass, generator="EPOSLHC", energy="14")